In [1]:
import numpy as np


import keras
from keras.layers import Conv2D, Conv2DTranspose, Input, UpSampling2D
from keras.models import Sequential

import keras.backend as K

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Creating a 4-D array to represent the input. The 4-day array is structured as ( image number, height, width, num of channels ) 

In [2]:
a = np.diag((5,5,5,5,5)).reshape((5,5,1))
b = np.diag((1,1,1,1,1)).reshape((5,5,1))
x = np.array([a,b])
x.shape

(2, 5, 5, 1)

In [3]:
# Backend function to retreive the output of first layer
def get_layer_output(x,model,layer=0):
    
    layer_output = K.function([model.layers[layer].input],[model.layers[layer].output])
    return layer_output([x])

# Backend function to retreive the kernel weights of the first layer
def get_kernel_output(x,model,layer=0):
    
    kernel_output = K.function([model.layers[layer].input],[model.layers[layer].kernel])
    return kernel_output([x])

In [4]:
def multiplySum(x,y):
    
    out = []
    
    val = 0.0
    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            val += x[i][j]*y[i][j]
            
    return val

def getKernel(x,channel=0, kernel=0):
    return(x[0][:,:,channel, kernel])
    
def getOutput(x, kernel=0, inputImage = 0):
    return(x[0][inputImage, : , : , kernel])
    
    

In [5]:
def convolve(x, kernel, stride = (1,1),padding=None):
    
    
    if padding!=None:
        temp = np.zeros((x.shape[0]+2*padding[0] , x.shape[1]+2*padding[1]))
        temp[padding[0]:(padding[0]+x.shape[0]) , padding[1]:(padding[1]+x.shape[1])] = x
        x = temp
        print(x)
    
    
    kernel_size = kernel.shape[0]
    
    out_size = ((x.shape[0]  - kernel_size)/stride[0] + 1 , (x.shape[1]  - kernel_size)/stride[1] + 1)
    
    output = np.zeros(out_size)
    
    for i in range(0,out_size[0]):
        for j in range(0,out_size[1]):
            
            output[i][j] = multiplySum(x[i*stride[0]:(i*stride[0]+kernel_size) , j*stride[1]:(j*stride[1]+kernel_size)] , kernel )
            
    return np.array(output)
    
    

## Conv2D

### Basic Convolution

In [203]:
model = Sequential()
model.add(Conv2D(2,(3,3), input_shape=(5,5,1)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 3, 3, 2)           20        
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [205]:
"""
The output is a 4-d array where the structure of the array is ( image number, height, width, kernel )
So output[0,:,:,:] represents the outputs of the convolution of the kernels on the first image
Similarly, output[:,:,:,1] represent the outputs of the convolution of the images with the second kernel

"""
output = layer_output([x])
output[0].shape

(2, 3, 3, 2)

In [206]:
"""
The output of kernel_output is a 4-D array where the structure is ( height, width, channel, kernel number)

weight[:,:,:,0] represents the first kernel for the first channel.

If the input image had 3 channels, then weight shape will be (3,3,3,2). Therefore, even if we define only 1 kernel, 
if there n channels in the image, then n 3x3 matrices are created to represent the kernel. 

In other words, each kernel has different weight matrices for different input channel
"""

weight = kernel_output([x])
weight[0].shape

(3, 3, 1, 2)

In [213]:
x_final = x[0,:,:,0]

In [214]:
convolve(x_final,getKernel(weight, kernel=0))

array([[-0.15143082,  2.094208  ,  1.01403967],
       [-2.0393534 , -0.15143082,  2.094208  ],
       [ 0.16954944, -2.0393534 , -0.15143082]])

In [215]:
getOutput(output, kernel=0,inputImage=0)

array([[-0.15143076,  2.094208  ,  1.0140396 ],
       [-2.0393534 , -0.15143037,  2.094208  ],
       [ 0.16954947, -2.0393534 , -0.15143076]], dtype=float32)

### Convolution with strides

In [222]:
model = Sequential()
model.add(Conv2D(2,(3,3), strides=(2,2), input_shape=(5,5,1)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 2, 2, 2)           20        
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [227]:
output = get_layer_output(x,model)
output[0].shape

(2, 2, 2, 2)

In [229]:
weight = get_kernel_output(x,model)
weight[0].shape

(3, 3, 1, 2)

In [234]:
convolve(x_final,getKernel(weight, kernel=0), stride=(2,2))

array([[2.61574216, 0.        ],
       [0.        , 0.        ]])

In [235]:
x_final

array([[5, 0, 0, 0, 0],
       [0, 5, 0, 0, 0],
       [0, 0, 5, 0, 0],
       [0, 0, 0, 5, 0],
       [0, 0, 0, 0, 5]])

In [236]:
getKernel(weight,kernel=0)

array([[ 0.37873605, -0.21734086,  0.10111681],
       [ 0.39398727,  0.38138524, -0.14542866],
       [-0.13162675,  0.29388335, -0.23697285]], dtype=float32)

In [247]:
convolve(x_final,getKernel(weight, kernel=0), stride=(2,2))

array([[ 2.61574216, -0.65813377],
       [ 0.50558403,  2.61574216]])

In [243]:
getOutput(output,inputImage=0,kernel=0)

array([[ 2.6157422 , -0.65813375],
       [ 0.505584  ,  2.6157422 ]], dtype=float32)

In [248]:
convolve(x_final,getKernel(weight, kernel=1), stride=(2,2))

array([[-2.34536022,  1.31472543],
       [ 1.07642606, -2.34536022]])

In [249]:
getOutput(output,inputImage=0,kernel=1)

array([[-2.3453603,  1.3147254],
       [ 1.076426 , -2.3453603]], dtype=float32)

## Upsampling2D

In [252]:
model = Sequential()
model.add(UpSampling2D((2,2), input_shape=(5,5,1)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d_1 (UpSampling2 (None, 10, 10, 1)         0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [253]:
output = get_layer_output(x,model)

In [255]:
"""
Imagine every single cell in your input array being replaced by a new cell of size (2,2) { (2,2) because thats 
parameter that we have passed to UpSampling2D}.
"""
getOutput(output)

array([[5., 5., 0., 0., 0., 0., 0., 0., 0., 0.],
       [5., 5., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 5., 5., 0., 0., 0., 0., 0., 0.],
       [0., 0., 5., 5., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 5., 5., 0., 0., 0., 0.],
       [0., 0., 0., 0., 5., 5., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 5., 5., 0., 0.],
       [0., 0., 0., 0., 0., 0., 5., 5., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 5., 5.],
       [0., 0., 0., 0., 0., 0., 0., 0., 5., 5.]], dtype=float32)

In [256]:
x1 = np.array([[1,2,3],[4,5,6]]).reshape(1,2,3,1)

In [259]:
model = Sequential()
model.add(UpSampling2D((3,1), input_shape=(2,3,1)))
model.summary()
output = get_layer_output(x1,model)
getOutput(output)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d_3 (UpSampling2 (None, 6, 3, 1)           0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [4., 5., 6.],
       [4., 5., 6.],
       [4., 5., 6.]], dtype=float32)

In [24]:
# # def upsampleMatrix():

# y = np.diag([1,1,1])
# y = y.reshape(1,y.shape[0],y.shape[1],1)
# y

# model = Sequential()
# model.add(UpSampling2D((factor,factor), input_shape=(y.shape[1],y.shape[2], y.shape[3])))



array([[[[1],
         [0],
         [0]],

        [[0],
         [1],
         [0]],

        [[0],
         [0],
         [1]]]])

## Conv2DTranspose

### Stride 1

#### same padding

In [6]:
x = np.diag((1,1,1)).reshape(1,3,3,1)
x_final=x[0,:,:,0]

In [328]:
model = Sequential()
model.add(Conv2DTranspose(1,(3,3), strides=(1,1),padding='same', input_shape=(3,3,1)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_10 (Conv2DT (None, 3, 3, 1)           10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [329]:
weight = get_kernel_output(x,model)
getKernel(weight)

array([[ 0.23676872,  0.23019093, -0.04622787],
       [-0.4418265 ,  0.11311585, -0.38515693],
       [ 0.36979067,  0.2140702 , -0.41140068]], dtype=float32)

In [330]:
output = get_layer_output(x,model)
getOutput(output)

array([[ 0.34988457, -0.154966  , -0.04622787],
       [-0.22775626, -0.06151611, -0.154966  ],
       [ 0.36979067, -0.22775626, -0.29828483]], dtype=float32)

In [333]:
convolve(x_final,getKernel(weight, kernel=0).T, stride=(1,1),padding=(1,1))

[[0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0.]]


array([[-0.29828483, -0.154966  , -0.04622787],
       [-0.22775629, -0.06151611, -0.154966  ],
       [ 0.36979067, -0.22775629,  0.34988457]])

In [342]:
getOutput(output)

array([[ 0.34988457, -0.154966  , -0.04622787],
       [-0.22775626, -0.06151611, -0.154966  ],
       [ 0.36979067, -0.22775626, -0.29828483]], dtype=float32)

#### valid padding

In [7]:
model = Sequential()
model.add(Conv2DTranspose(1,(3,3), strides=(1,1),padding='valid', input_shape=(3,3,1)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_1 (Conv2DTr (None, 5, 5, 1)           10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [8]:
weight = get_kernel_output(x,model)
getKernel(weight)

array([[ 0.00166929, -0.4668319 , -0.45973036],
       [ 0.38341302,  0.49846447, -0.33313805],
       [ 0.39259535,  0.44750214, -0.43990558]], dtype=float32)

In [9]:
output = get_layer_output(x,model)
getOutput(output)

array([[ 0.00166929, -0.46683192, -0.4597304 ,  0.        ,  0.        ],
       [ 0.38341302,  0.50013375, -0.79997   , -0.45973036,  0.        ],
       [ 0.39259535,  0.83091515,  0.06022817, -0.79997   , -0.4597304 ],
       [ 0.        ,  0.39259535,  0.83091515,  0.05855885, -0.33313805],
       [ 0.        ,  0.        ,  0.39259535,  0.44750214, -0.43990558]],
      dtype=float32)

In [10]:
def deconvolve(x,kernel,stride = 1):
    
    out_size = x.shape[0]*stride + max( (kernel.shape[0]-stride) , 0 )
    
    output = np.zeros((out_size,out_size))
    
    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            
            val = kernel*x[i][j]
            
            output[i*stride:(i*stride+kernel.shape[0]) , j*stride:(j*stride+kernel.shape[0])] += val
            
    return output
    

In [11]:
deconvolve(x_final,getKernel(weight, kernel=0))

array([[ 0.00166929, -0.46683189, -0.45973036,  0.        ,  0.        ],
       [ 0.38341302,  0.50013375, -0.79996994, -0.45973036,  0.        ],
       [ 0.39259535,  0.83091515,  0.06022817, -0.79996994, -0.45973036],
       [ 0.        ,  0.39259535,  0.83091515,  0.05855888, -0.33313805],
       [ 0.        ,  0.        ,  0.39259535,  0.44750214, -0.43990558]])

In [12]:
"""
Comparing the output of convolve with padding (1,1) with the output of deconvolve, we can see that the 
central 3x3 region of the 5x5 output of deconvolve corresponds to the output of convolve. Therefore, to go from 
"valid" padding to "same", we can just crop down the edges of the output of "valid" padding
"""
convolve(x_final,getKernel(weight, kernel=0), stride=(1,1),padding=(1,1))

[[0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0.]]


array([[ 0.05855888,  0.83091515,  0.39259535],
       [-0.79996994,  0.06022817,  0.83091515],
       [-0.45973036, -0.79996994,  0.50013375]])

In [21]:
convolve(x_final,np.flip(np.flip(getKernel(weight, kernel=0),0),1), stride=(1,1),padding=(1,1))

[[0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0.]]


array([[ 0.50013375, -0.79996994, -0.45973036],
       [ 0.83091515,  0.06022817, -0.79996994],
       [ 0.39259535,  0.83091515,  0.05855888]])

### Stride 2

#### valid padding

In [26]:
model = Sequential()
model.add(Conv2DTranspose(1,(3,3), strides=(2,2),padding='valid', input_shape=(3,3,1)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_2 (Conv2DTr (None, 7, 7, 1)           10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [27]:
weight = get_kernel_output(x,model)
getKernel(weight)

array([[-0.42253345,  0.34132046, -0.32057053],
       [-0.3672353 , -0.2650267 ,  0.30951786],
       [ 0.26088667, -0.5487001 ,  0.49119878]], dtype=float32)

In [28]:
output = get_layer_output(x,model)
getOutput(output)

array([[-0.42253345,  0.34132046, -0.32057053,  0.        ,  0.        ,
         0.        ,  0.        ],
       [-0.3672353 , -0.2650267 ,  0.30951786,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.26088667, -0.5487001 ,  0.06866533,  0.34132046, -0.32057053,
         0.        ,  0.        ],
       [ 0.        ,  0.        , -0.3672353 , -0.2650267 ,  0.30951786,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.26088667, -0.5487001 ,  0.06866533,
         0.34132046, -0.32057053],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0.3672353 ,
        -0.2650267 ,  0.30951786],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.26088667,
        -0.5487001 ,  0.49119878]], dtype=float32)

In [35]:
np.matrix.round(deconvolve(x_final,getKernel(weight, kernel=0),stride=2),3)

array([[-0.423,  0.341, -0.321,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.367, -0.265,  0.31 ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.261, -0.549,  0.069,  0.341, -0.321,  0.   ,  0.   ],
       [ 0.   ,  0.   , -0.367, -0.265,  0.31 ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.261, -0.549,  0.069,  0.341, -0.321],
       [ 0.   ,  0.   ,  0.   ,  0.   , -0.367, -0.265,  0.31 ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.261, -0.549,  0.491]])

### Checking for equivalance between Conv2DTranpose and ( Upsampling + Conv2D)

In [37]:
y = np.array([[1,0,0,0,0,0],
             [0,0,0,0,0,0],
             [0,0,1,0,0,0],
             [0,0,0,0,0,0],
             [0,0,0,0,1,0],
             [0,0,0,0,0,0]])

In [38]:
np.matrix.round(convolve(y,np.flip(np.flip(getKernel(weight, kernel=0),0),1),padding=(1,1)),3)

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


array([[-0.265,  0.31 ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.549,  0.069,  0.341, -0.321,  0.   ,  0.   ],
       [ 0.   , -0.367, -0.265,  0.31 ,  0.   ,  0.   ],
       [ 0.   ,  0.261, -0.549,  0.069,  0.341, -0.321],
       [ 0.   ,  0.   ,  0.   , -0.367, -0.265,  0.31 ],
       [ 0.   ,  0.   ,  0.   ,  0.261, -0.549,  0.491]])

In [40]:
np.matrix.round(deconvolve(x_final,getKernel(weight, kernel=0),stride=2),3)[1:7,1:7]

array([[-0.265,  0.31 ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.549,  0.069,  0.341, -0.321,  0.   ,  0.   ],
       [ 0.   , -0.367, -0.265,  0.31 ,  0.   ,  0.   ],
       [ 0.   ,  0.261, -0.549,  0.069,  0.341, -0.321],
       [ 0.   ,  0.   ,  0.   , -0.367, -0.265,  0.31 ],
       [ 0.   ,  0.   ,  0.   ,  0.261, -0.549,  0.491]])